This notebook was inspired by neural network & machine learning labs led by [GMUM](https://gmum.net/).

See also [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) and [Chapter 10](https://www.deeplearningbook.org/contents/rnn.html) of the Deep Learning book.

Some utils for today's class:

In [1]:
import os
import unicodedata
import string
from itertools import chain
from typing import Tuple, Optional, List

import numpy as np
import matplotlib.pyplot as plt 
from sklearn.metrics import f1_score

import torch
from torch.nn.functional import cross_entropy
from torch.utils.data import Dataset, DataLoader

all_letters = string.ascii_letters
n_letters = len(all_letters)


class ListDataset(Dataset):
    
    def __init__(self, data, targets):
        
        self.data = data
        self.targets = targets
        
    def __getitem__(self, ind):
        
        return self.data[ind], self.targets[ind]
    
    def __len__(self):
        return len(self.targets)

    
def unicode_to__ascii(s: str) -> str:
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn'
                                                                 and c in all_letters)
                   

def read_lines(filename: str) -> List[str]:
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicode_to__ascii(line) for line in lines]


def letter_to_index(letter: str) -> int:
    return all_letters.find(letter)


def line_to_tensor(line: str) -> torch.Tensor:
    tensor = torch.zeros(len(line), n_letters)
    for i, letter in enumerate(line):
        tensor[i][letter_to_index(letter)] = 1
    return tensor

# Recurrent neural networks

The models we've used so far have no concept of memory. This can be a major shortcoming in certain contexts and might be an inductive bias we want to include. This is visible especially in the case of sequential data:
* natural language,
* time series,
* sound.

Recurrent neural networks are one way of dealing with the above. They have loops, allowing information to persist.

![layer based](figures/unfold.png)
<center>Source: <a href="https://www.deeplearningbook.org/contents/rnn.html">Chapter 10 </a>of the Deep Learning book.</center>

Today we'll be working with a simple prediction task (by Sean Robertson). The cell below downloads a dataset of names of 18 different nationalities. Each letter in a name is changed into the so-called *one-hot encoding* and in the end each name is an array of shape `(n_letters, len(name))` consisting of zeroes and ones. 

We'll also use a sampler because the dataset is quite imbalanced and we want the network to see the same amount of examples from each class.

Because the names can have different lengths we'll use `batch_size=1` in this notebook (though the model implementations should work for any values).

In [2]:
!wget https://download.pytorch.org/tutorial/data.zip
!unzip data.zip

--2021-05-24 16:05:18--  https://download.pytorch.org/tutorial/data.zip
Resolving download.pytorch.org (download.pytorch.org)... 2600:9000:20ae:7a00:d:607e:4540:93a1, 2600:9000:20ae:1000:d:607e:4540:93a1, 2600:9000:20ae:f200:d:607e:4540:93a1, ...
Connecting to download.pytorch.org (download.pytorch.org)|2600:9000:20ae:7a00:d:607e:4540:93a1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2882130 (2,7M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]   2,75M  3,87MB/s    in 0,7s    

2021-05-24 16:05:19 (3,87 MB/s) - ‘data.zip’ saved [2882130/2882130]

Archive:  data.zip
   creating: data/
  inflating: data/eng-fra.txt        
   creating: data/names/
  inflating: data/names/Arabic.txt   
  inflating: data/names/Chinese.txt  
  inflating: data/names/Czech.txt    
  inflating: data/names/Dutch.txt    
  inflating: data/names/English.txt  
  inflating: data/names/French.txt   
  inflating: data/names/German.txt   
  inflatin

In [3]:
data_dir = 'data/names'

category_lines = {}
all_categories = []

data = []
targets = [] 
label_to_idx = {}

for label, file_name in enumerate(os.listdir(data_dir)):
    
    label_to_idx[label] = file_name.split('.')[0].lower()
    
    names = read_lines(os.path.join(data_dir, file_name))
    data += [line_to_tensor(name) for name in names]
    targets += len(names) * [label]

test_frac = 0.1
n_test = int(test_frac * len(targets))
test_ind = np.random.choice(len(targets), size=n_test, replace=False)
train_ind = np.setdiff1d(np.arange(len(targets)), test_ind)

targets = torch.tensor(targets)
train_targets = targets[train_ind]

uni, counts = np.unique(train_targets, return_counts=True)
weight_per_class = len(targets) / counts
weight = [weight_per_class[c] for c in train_targets]

sampler = torch.utils.data.sampler.WeightedRandomSampler(weights=weight, num_samples=len(weight)) 

train_dataset = ListDataset(data=[x for i, x in enumerate(data) if i in train_ind], targets=train_targets)
train_loader = DataLoader(train_dataset, shuffle=False, batch_size=1, sampler=sampler)

test_dataset = ListDataset(data=[x for i, x in enumerate(data) if i in test_ind], targets=targets[test_ind])
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=1)

The cell below shows an example datapoint.

In [4]:
x, y = next(iter(train_loader))

print("x.shape:", x.shape)
print("name: ", end="")
for letter_onehot in x[0]:
    print(all_letters[torch.argmax(letter_onehot)], end="")

print("\ny:", label_to_idx[y.item()])

x.shape: torch.Size([1, 3, 52])
name: Gil
y: korean


## Task 1 (0.5p)
Implement a basic recurrent neural network.

![layer based](figures/LSTM3-SimpleRNN.png)
<center>Source: <a href="https://colah.github.io/posts/2015-08-Understanding-LSTMs/">Understanding LSTM Networks</a>.</center>

Some remarks:
* Define the needed layers and implement the basic logic for one timestep in the `RNN` class.
* The output can be of any submitted size, hence you'll need to add a fully-connected layer at the end.
* You'll need to iterate over the time dimension in the training loop.

In [5]:
class RNN(torch.nn.Module):
    
    def __init__(self, 
                 input_size: int,
                 hidden_size: int, 
                 output_size: int):
        """
        :param input_size: int
            Dimensionality of the input vector
        :param hidden_size: int
            Dimensionality of the hidden space
        :param output_size: int
            Desired dimensionality of the output vector
        """
        super(RNN, self).__init__()

        self.hidden_size = hidden_size
        self.input_to_hidden = torch.nn.Linear(in_features=input_size+hidden_size, out_features=hidden_size)  
        self.hidden_to_output = torch.nn.Linear(in_features=hidden_size, out_features=output_size)
    
    # for the sake of simplicity the forward pass will process a single timestep 
    def forward(self, 
                input: torch.tensor, 
                hidden: torch.tensor) -> Tuple[torch.tensor, torch.tensor]:
        """
        :param input: torch.tensor 
            Input tensor for a single observation at a given timestep
            shape [batch_size, input_size]
        :param hidden: torch.tensor
            Representation of the memory of the RNN from the previous timestep
            shape [batch_size, hidden_size]
        """

        combined = torch.cat([input, hidden], dim=1) 
        hidden = torch.tanh(self.input_to_hidden(combined))
        output = self.hidden_to_output(hidden)
        return output, hidden
    
    def init_hidden(self, batch_size: int) -> torch.Tensor:
        """
        Returns the initial value for the hidden state
        """
        return torch.zeros(batch_size, self.hidden_size, requires_grad=True)

In [6]:
n_class = len(label_to_idx)

rnn = RNN(n_letters, 256, n_class)
optimizer = torch.optim.SGD(rnn.parameters(), lr=0.01)   

n_epochs = 1

for epoch in range(n_epochs):
    
    loss_buffer = []
    
    for i, (x, y) in enumerate(train_loader):  
        
        optimizer.zero_grad()
        
        # get initial hidden state
        hidden = rnn.init_hidden(x.shape[0])
        
        # iterate over the time dimension
        seq_len = x.shape[1]      
        for t in range(seq_len):
            # define output here
            x_t = x[:, t]
            output, hidden = rnn(input=x_t, hidden=hidden)
            
        loss = cross_entropy(output, y)
        loss.backward()
        optimizer.step()  
        
        loss_buffer.append(loss.item())
        
        if i % 1000 == 1:
            print(f"\rEpoch: {epoch+1} Progress: {100 * i/len(train_loader):2.0f}% Loss: {np.mean(loss_buffer):.3f}", end="")
            loss_buffer = []
    

# evaluate on the test set
with torch.no_grad():
    ps = []
    ys = []
    correct = 0
    for i, (x, y) in enumerate(test_loader):
        ys.append(y.numpy())

        hidden = rnn.init_hidden(x.shape[0])
        
        seq_len = x.shape[1]
        for t in range(seq_len):
            # define output here
            x_t = x[:, t]
            output, hidden = rnn(input=x_t, hidden=hidden)

        pred = output.argmax(dim=1)
        ps.append(pred.numpy())
    
    ps = np.concatenate(ps, axis=0)
    ys = np.concatenate(ys, axis=0)
    f1 = f1_score(ys, ps, average='weighted')
    
    print(f"\nFinal F1 score: {f1:.2f}")
    assert f1 > 0.15, "You should get f1 score over 0.15"

/home/kg/miniconda3/envs/aann/lib/python3.8/site-packages/torch/autograd/__init__.py:130: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1607369981906/work/c10/cuda/CUDAFunctions.cpp:100.)
  Variable._execution_engine.run_backward(


Epoch: 1 Progress: 100% Loss: 1.658
Final F1 score: 0.20


## Task 2 (0.25p)
Implement the `predict` function, which takes in a name and a RNN model and outputs the top 3 nationality predictions for that name, as well as their logits.

Hint: use one of the functions from the beginning of the notebook and [`torch.topk`](https://pytorch.org/docs/stable/generated/torch.topk.html).

In [7]:
def predict(name: str, rnn: RNN):
    x = line_to_tensor(name).unsqueeze(0)

    hidden = rnn.init_hidden(x.shape[0])
    seq_len = x.shape[1]
    for t in range(seq_len): 
        x_t = x[:, t]
        output, hidden = rnn(input=x_t, hidden=hidden)

    res = output.topk(3, dim=1)
 
    for score, ind in zip(res.values[0], res.indices[0]):
        print(f"\t{label_to_idx[ind.item()]}: {score:.2f}")

In [8]:
some_names = ["Satoshi", "Jackson", "Schmidhuber", "Hinton", "Kowalski"]

for name in some_names:
    print(name)
    predict(name, rnn)

Satoshi
	japanese: 4.42
	arabic: 2.73
	polish: 2.66
Jackson
	scottish: 3.60
	dutch: 2.33
	english: 2.31
Schmidhuber
	german: 3.05
	arabic: 2.97
	dutch: 2.58
Hinton
	scottish: 2.95
	russian: 2.09
	polish: 2.06
Kowalski
	polish: 6.51
	russian: 2.99
	japanese: 2.76


## Task 3 (0.75p)
Implement the LSTM network.

![layer based](figures/LSTM3-chain.png)
<center>Source: <a href="https://colah.github.io/posts/2015-08-Understanding-LSTMs/">Understanding LSTM Networks</a>.</center>

* The `LSTMCell` class should contain the needed layers for the LSTM gates.
* The `LSTM` class should use the `LSTMCell` class. The iteration over timesteps that was previously done in the training loop should now be moved here.

In [9]:
class LSTMCell(torch.nn.Module):

    def __init__(self, 
                 input_size: int, 
                 hidden_size: int):
        """
        :param input_size: int
            Dimensionality of the input vector
        :param hidden_size: int
            Dimensionality of the hidden space
        """   
        super(LSTMCell, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size

        # initialize LSTM weights 
        self.W_f = torch.nn.Linear(in_features=hidden_size+input_size, out_features=hidden_size)
        self.W_i = torch.nn.Linear(in_features=hidden_size+input_size, out_features=hidden_size)
        self.W_c = torch.nn.Linear(in_features=hidden_size+input_size, out_features=hidden_size)
        self.W_o = torch.nn.Linear(in_features=hidden_size+input_size, out_features=hidden_size)

    def forward(self, 
                input: torch.tensor, 
                states: Tuple[torch.tensor, torch.tensor]) -> Tuple[torch.tensor, torch.tensor]:
        
        hidden, cell = states  
        combined = torch.cat([input, hidden], dim=1)
        
        # compute input, forget, and output gates       
        forget_gate = torch.sigmoid(self.W_f(combined))
        input_gate = torch.sigmoid(self.W_i(combined))
        output = torch.sigmoid(self.W_o(combined))

        # compute new cell state and hidden state
        cell_update = torch.tanh(self.W_c(combined))
        cell = forget_gate * cell + input_gate * cell_update  
        hidden = output * torch.tanh(cell)
        
        return hidden, cell

In [10]:
class LSTM(torch.nn.Module):

    def __init__(self, 
                 input_size: int, 
                 hidden_size: int):
        """
        :param input_size: int
            Dimensionality of the input vector
        :param hidden_size: int
            Dimensionality of the hidden space
        """
        super(LSTM, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size

        self.cell = LSTMCell(input_size=input_size, hidden_size=hidden_size)
        
    def forward(self, 
                input: torch.tensor) -> Tuple[torch.tensor, torch.tensor]:
        """
        :param input: torch.tensor
            Input tensor for a single observation 
            shape [batch_size, seq_len, input_size]
        Returns tuple of two torch.tensors, both of shape [seq_len, batch_size, hidden_size]
        """
        
        batch_size = input.shape[0]
        
        hidden, cell = self.init_hidden_cell(batch_size)
                
        hiddens = []
        cells = []
        
        # process the whole sequence in the forward method (as opposed to the previous exercise)        
        seq_len = input.shape[1]
        for t in range(seq_len):
            x = input[:, t]
            hidden, cell = self.cell(x, (hidden, cell))
            
            hiddens.append(hidden)
            cells.append(cell)

        return torch.stack(hiddens), torch.stack(cells)
    
    def init_hidden_cell(self, batch_size) -> Tuple[torch.tensor, torch.tensor]:
        """
        Returns the initial value for the hidden and cell states
        """
        return (torch.zeros(batch_size, self.hidden_size, requires_grad=True), 
                torch.zeros(batch_size, self.hidden_size, requires_grad=True))

In [11]:
# initialize the LSTM network with an additional classifier layer on top
lstm = LSTM(input_size=len(all_letters), hidden_size=128)
clf = torch.nn.Linear(in_features=128, out_features=len(label_to_idx))

params = chain(lstm.parameters(), clf.parameters())
optimizer = torch.optim.Adam(params, lr=0.01) 

n_epochs = 1

for epoch in range(n_epochs):
    
    loss_buffer = []
    
    for i, (x, y) in enumerate(train_loader):   
        
        optimizer.zero_grad()
        
        hidden, _ = lstm(x)
        output = clf(hidden[-1])

        loss = cross_entropy(output, y)
        loss.backward()
        optimizer.step()                                
        
        loss_buffer.append(loss.item())
        
        if i % 1000 == 1:
            print(f"\rEpoch: {epoch+1} Progress: {100 * i/len(train_loader):2.0f}% Loss: {np.mean(loss_buffer):.3f}", end="")
            loss_buffer = []

# evaluate on the test set
with torch.no_grad():
    
    ps = []
    ys = []
    for i, (x, y) in enumerate(test_loader): 
        
        ys.append(y.numpy())
        
        hidden, _ = lstm(x)
        output = clf(hidden[-1])

        pred = output.argmax(dim=1)
        ps.append(pred.numpy())
    
    ps = np.concatenate(ps, axis=0)
    ys = np.concatenate(ys, axis=0)
    f1 = f1_score(ys, ps, average='weighted')
    
    print(f"\nFinal F1 score: {f1:.2f}")
    assert f1 > 0.18, "You should get f1 score over 0.18"

Epoch: 1 Progress: 100% Loss: 0.910
Final F1 score: 0.21


## Task 4 (0.25p)
Implement the `predict` function for the LSTM+CLF model.

In [12]:
def predict_lstm(name: str, lstm: LSTM, clf: torch.nn.Module):
    x = line_to_tensor(name).unsqueeze(0)

    hidden, _ = lstm(input=x)
    output = clf(hidden[-1])

    res = output.topk(3, dim=1)
 
    for score, ind in zip(res.values[0], res.indices[0]):
        print(f"\t{label_to_idx[ind.item()]}: {score:.2f}")

In [13]:
some_names = ["Satoshi", "Jackson", "Schmidhuber", "Hinton", "Kowalski"]
    
for name in some_names:
    print(name)
    predict_lstm(name, lstm, clf)

Satoshi
	japanese: 3.76
	italian: 0.33
	greek: -0.47
Jackson
	scottish: 5.12
	english: 2.06
	dutch: -0.83
Schmidhuber
	german: 4.48
	czech: 3.35
	russian: 2.93
Hinton
	scottish: 2.30
	english: 1.83
	dutch: -0.66
Kowalski
	polish: 4.98
	japanese: 1.19
	russian: 0.51
